In [2]:
import os, cv2, tqdm
import copy
from tqdm import tqdm

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import detection_utils as utils
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def MyMapper(dataset_dict):
    
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    dataset_dict['image'] = image
    
    return dataset_dict


In [4]:
try:
    register_coco_instances('coco_trash_test', {}, '../dataset/test.json', '../dataset/')
except AssertionError:
    pass

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))
TrashMeta = MetadataCatalog.get("coco_trash_test")
# config 수정하기
cfg.DATASETS.TEST = ('coco_trash_test',)
cfg.DATALOADER.NUM_WOREKRS = 2
cfg.OUTPUT_DIR = './output'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

predictor = DefaultPredictor(cfg)
test_loader = build_detection_test_loader(cfg, 'coco_trash_test', MyMapper)

BreakCounter = 100
for data in tqdm(test_loader):
    
    data = data[0]
    im = cv2.imread(data['file_name'])
    outputs = predictor(data['image'])['instances']
    output = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                    metadata = TrashMeta,
                    scale=0.5,
                    instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
                    )
    out = v.draw_instance_predictions(output["instances"].to("cpu"))
    os.makedirs('output/validation', exist_ok=True)
    #fdir=data['file_name'].replace('../dataset/test','output/validation')
    #cv2.imwrite(fdir, out.get_image()[:, :, ::-1])
    ImageId = data['file_name'].replace('../dataset/test','')
    cv2.imshow(ImageId, out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    
    
    if BreakCounter == 0:
        break
    else:
        BreakCounter=BreakCounter-1

  0%|          | 0/4871 [00:00<?, ?it/s]/data/ephemeral/home/level2-objectdetection-cv-09/detectron2/structures/image_list.py:99: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


: 